In [ ]:
from numpy.random import seed

max_sent_tokens = 75
n_folds = 5

seed(123456)

In [ ]:
import os
import re
import us
import sys
import json

from nltk import sent_tokenize
from nltk import word_tokenize
from random import shuffle

data_path = '/media/rbshaffer/My Paasport/Current_Materials/dissertation/Legislative_Data/Legislation/UnitedStates/Annual'

file_list = os.listdir(data_path)

sys.path.append('/home/rbshaffer/PycharmProjects/Legislative_Data/')
from _country_entities_annual import UnitedStates

blacklist = ['Guam', 'Federated States of Micronesia','American Samoa', 
             'Puerto Rico', 'Virgin Islands',  'Northern Mariana Islands', 
             'Republic of The Marshall Islands', ]

In [ ]:
with open('/home/rbshaffer/PycharmProjects/Legislative_Data/agency_list.txt', 'r') as f:
    agency_list = f.read().split('\n')
    
agency_list = [e for e in agency_list if e and e not in [s.name for s in us.states.STATES]]
agency_list = [e for e in agency_list if e not in blacklist]

In [ ]:
def process_inds(inst_list, sentence_str, tokens_list, label, tag_set, label_list):
    for inst_str in inst_list:
        if inst_str + ' ' in sentence_str or ' ' + inst_str in sentence_str:
            entity_len = len(inst_str.split())
            match_inds = [i for i in range(len(tokens_list)) 
                          if tokens_list[i:i + entity_len] == inst_str.split()]
            for i in match_inds:
                if i not in tag_set:
                    tag_set[i] = 'B-MISC'
                    
                    label_list.append(label)
                if entity_len > 1:
                    for j in range(i+1, i+entity_len):
                        if j not in tag_set:
                            tag_set[j] = 'I-MISC'
    
    return tag_set, label_list

In [ ]:
from numpy import random

fold_assignments = random.choice(range(n_folds), len(agency_list))

train_sets = {i: {'names': [agency_list[j] for j in range(len(agency_list)) if fold_assignments[j] != i],
                  'sents': []}
              for i in range(n_folds)}

test_sets = {i: {'names': [agency_list[j] for j in range(len(agency_list)) if fold_assignments[j] == i],
                 'sents': []}
              for i in range(n_folds)}

full_set = []

for file_name in file_list:
    print(re.sub('_', '/', file_name).strip('.json'))

    with open(data_path + '/' + file_name, 'r') as f:
        content = json.loads(f.read())

        if content['parsed']:
            # imported from system path above
            parser = UnitedStates(content['parsed'], load_lstm=False)
            for chunk in parser.chunks:
                sentences = sent_tokenize(chunk)
                for sentence in sentences:
                    sentence = re.sub('\s+', ' ', sentence)
                    tokens = word_tokenize(sentence)
                    
                    for i in range(n_folds):
                        tags = {}
                        labels = []
                        
                        tags, labels = process_inds(train_sets[i]['names'], sentence, tokens, 'train', tags, labels)
                        tags, labels = process_inds(test_sets[i]['names'], sentence, tokens, 'test', tags, labels)     
                    
                        tags.update({j: 'O' for j in range(len(tokens)) if j not in tags.keys()})
                        to_append = '\n'.join(tokens[j] + ' ' + tags[j] for j in range(len(tokens)))

                        if 'train' in labels and 'test' not in labels:
                            train_sets[i]['sents'].append(to_append)
                        elif 'test' in labels and 'train' not in labels:
                            test_sets[i]['sents'].append(to_append)
                            
                    if 'test' in labels or 'train' in labels:
                        full_set.append(to_append)      
                    

In [ ]:
with open('/home/rbshaffer/sequence_tagging/data/data_v2_full.txt', 'w') as f:
    f.write('\n\n'.join([e for e in full_set if len(e.split()) < max_sent_tokens and 'MISC' in e]))

In [ ]:
for i in range(n_folds):
    train_list = [e for e in train_sets[i]['sents'] if len(e.split()) < max_sent_tokens and 'MISC' in e]
    shuffle(train_list)

    test_list = [e for e in test_sets[i]['sents'] if len(e.split()) < max_sent_tokens and 'MISC' in e]
    shuffle(test_list)

    with open('/home/rbshaffer/sequence_tagging/data/data_v2_' + str(i) + '_train.txt', 'w') as f:
        f.write('\n\n'.join(train_list))

    with open('/home/rbshaffer/sequence_tagging/data/data_v2_' + str(i) + '_test.txt', 'w') as f:
        f.write('\n\n'.join(test_list))